In [37]:
from tweepy import Stream
import pandas as pd
import numpy as np
import tweepy
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
import argparse
import string
from twython import Twython  
import json
from sklearn import preprocessing
import gensim 
import re, string
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import re
import os
import sys

In [38]:
def get_latest_tweets(data_dir, auth, time_limit, topic):
    file_name = "stream"
    twitter_stream = Stream(auth, MyListener(data_dir, topic, time_limit),tweet_mode='extended')
    twitter_stream.filter(track= topic) # list of querries to track
    data = pd.read_json(data_dir+file_name+".json",lines=True)
    tweets=pd.DataFrame(columns=['time','tweet'],index=data.index)
    no_data = False
    if data.empty:
        no_data = True
    else:
        data = data[data.lang=='en']
        #To get the full-text of the tweet
        for i in data.index:  
               tweets.time[i] = data.created_at[i]
               if pd.isnull(data.retweeted_status[i]):
                     if pd.isnull(data.extended_tweet[i]):
                            tweets.tweet[i] = data.text[i]
                     else:   
                        if "full_text" in data.extended_tweet[i].keys():
                             tweets.tweet[i]=data.extended_tweet[i]["full_text"]

                        else:
                             tweets.tweet[i]=data.text[i] 
               else:
                    if 'extended_tweet' in data.retweeted_status[i].keys():
                        if "full_text" in data.retweeted_status[i]['extended_tweet'].keys():
                            tweets.tweet[i]= data.retweeted_status[i]['extended_tweet']["full_text"]
                    else:
                         tweets.tweet[i] = data.retweeted_status[i]['text']     
        tweets = tweets.sort_values('time', ascending=False)
        tweets=tweets.drop_duplicates()
        tweets.dropna(subset=['tweet'])
    return tweets, no_data

    

In [39]:
#MyListener() saves the data into a .json file with name stream_query
class MyListener(StreamListener):
    """Custom StreamListener for streaming data."""

    def __init__(self, data_dir, query, time_limit=60):
        self.start_time = time.time()
        self.limit = time_limit
        #query_fname = format_filename(query)
        self.saveFile = open(data_dir+"stream.json", 'a')
        super(MyListener, self).__init__()

    def on_data(self, data):
        if (time.time() - self.start_time) < self.limit:
            self.saveFile.write(data)
            return True
        else:
            self.saveFile.close()
            return False
            

    def on_error(self, status):
        print(status)
        return True


In [40]:
def format_filename(fname):
    """Convert file name into a safe string.
    Arguments:
        fname -- the file name to convert
    Return:
        String -- converted file name
    """
    return ''.join(convert_valid(one_char) for one_char in fname)


def convert_valid(one_char):
    """Convert a character into '_' if invalid.
    Arguments:
        one_char -- the char to convert
    Return:
        Character -- converted char
    """
    valid_chars = "-_.%s%s" % (string.ascii_letters, string.digits)
    if one_char in valid_chars:
        return one_char
    else:
        return '_'

In [41]:
import spacy
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
from contractions import CONTRACTION_MAP
import unicodedata

class TwitterPreprocessor:
    def __init__(self, text: str):
        self.text = text
    
    tokenizer = ToktokTokenizer()
    stopword_list = nltk.corpus.stopwords.words('english')
    stopword_list.remove('no')
    stopword_list.remove('not')  
    
    def fully_preprocess(self):
        return self \
            .remove_hyphens() \
            .remove_urls() \
            .remove_mentions() \
            .remove_hashtags() \
            .remove_twitter_reserved_words() \
            .remove_single_letter_words() \
            .remove_stopwords() \
            .remove_numbers() \
            .strip_html_tags() \
            .remove_accented_chars() \
            .expand_contractions() \
            .lemmatize_text() \
            .remove_special_characters() \
            .remove_stopwords() \
            .remove_blank_spaces() \
            .Lower()
    
    def Lower(self):
        self.text = self.text.lower()
        return self
    def strip_html_tags(self):
        soup = BeautifulSoup(self.text, "html.parser")
        self.text = soup.get_text()
        return self
    
    def remove_accented_chars(self):
        self.text = unicodedata.normalize('NFKD', self.text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        return self
    
    def expand_contractions(self):
        contractions_pattern = re.compile('({})'.format('|'.join(CONTRACTION_MAP.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
        def expand_match(contraction):
            match = contraction.group(0)
            first_char = match[0]
            expanded_contraction = CONTRACTION_MAP.get(match) \
                                   if CONTRACTION_MAP.get(match) \
                                    else CONTRACTION_MAP.get(match.lower())                       
            expanded_contraction = first_char+expanded_contraction[1:]
            return expanded_contraction
        
        expanded_text = contractions_pattern.sub(expand_match, self.text)
        self.text = re.sub("'", "", expanded_text)
        return self
    
# # Removing Special Characters
    def remove_special_characters(self):
        self.text = re.sub('[^a-zA-Z0-9\s]', ' ', self.text)
        return self


# # Lemmatizing text
    def lemmatize_text(self):
        nlp = spacy.load('en', parse = False, tag=False, entity=False)
        text = nlp(self.text)
        self.text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
        return self
# # Removing Stopwords
    def remove_stopwords(self):
        tokens = tokenizer.tokenize(self.text)
        tokens = [token.strip() for token in tokens]
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
        self.text = ' '.join(filtered_tokens)    
        return self
    
    def remove_hyphens(self):
        self.text=self.text.replace("-"," ")
        return self
    def remove_urls(self):
        self.text = re.sub(pattern=re.compile(
        r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))'
        r'[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})'), repl='', string=self.text)
        return self

#     def remove_punctuation(self):
#         self.text = self.text.translate(str.maketrans('', '', string.punctuation))
#         return self
 
    def remove_mentions(self):
        self.text = re.sub(pattern=re.compile(r'@\w*'), repl='', string=self.text)
        return self

    def remove_hashtags(self):
        self.text = re.sub(pattern=re.compile(r'#*'),repl='',string=self.text)
        return self
    def remove_twitter_reserved_words(self):
        self.text = re.sub(r'\brt\b','',self.text,flags=re.IGNORECASE) #removing rt
        self.text = re.sub(r'\bvia\b','',self.text,flags=re.IGNORECASE) #removing via
        return self

    def remove_single_letter_words(self):
        self.text = re.sub(pattern=re.compile(r'(?<![\w\-])\w(?![\w\-])'),repl=' ',string=self.text)
        return self
    def remove_stopwords(self, extra_stopwords=None):
        if extra_stopwords is None:
            extra_stopwords = []
        text = nltk.word_tokenize(self.text)
        stop_words = set(stopwords.words('english'))
        new_sentence = []
        for w in text:
            if w not in stop_words and w not in extra_stopwords:
                new_sentence.append(w)
        self.text = ' '.join(new_sentence)
        return self
    def remove_numbers(self, preserve_years=False):
        text_list = self.text.split(' ')
        for text in text_list:
            if text.isnumeric():
                if preserve_years:
                    if utils.is_year(text):
                        text_list.remove(text)
                else:
                    text_list.remove(text)

        self.text = ' '.join(text_list)
        return self
    def remove_blank_spaces(self):
        self.text = re.sub(pattern=re.compile(r'\s{2,}|\t'),repl='',string=self.text)
        return self


In [42]:
def vectorize_latest_tweets(tweets):
    tweets.tweet= tweets.tweet.astype(str)
    tweets.loc[:,'Normalized'] = tweets.tweet.apply(lambda x:TwitterPreprocessor(x).fully_preprocess().text)
    tweets.loc[:,'Normalized'] = [tweet.split() for tweet in tweets.Normalized]
    vectorized_tweets = pd.DataFrame(columns=["Raw_tweet","Normalized_tweet","Vector"])

    for index, tweet in tweets.iterrows():
        word_vecs = [model[word] for word in (tweet.Normalized and model.vocab)]
        if len(word_vecs) == 0:
            tweet_vec = [np.zeros(300)]
        else: 
            tweet_vec = preprocessing.normalize(sum(word_vecs).reshape(1, -1))
        vectorized_tweets = vectorized_tweets.append \
        ({'Raw_tweet': tweet.tweet, 'Normalized_tweet': tweet.Normalized,'Vector': tweet_vec}, ignore_index=True)
    return vectorized_tweets

In [43]:
def vectorize_user_input(user_input):
    
    Normalized = TwitterPreprocessor(user_input).fully_preprocess().text.split()
    word_vecs = [model[word] for word in (Normalized and model.vocab)]
    if len(word_vecs) == 0:
        tweet_vec = [np.zeros(300)] #300 is the length of the vector model[word]
    else:
        tweet_vec = preprocessing.normalize(sum(word_vecs).reshape(1, -1))
    vectorized_input = {'Raw_input': user_input, 'Normalized': Normalized, 'Vector': tweet_vec}
    return vectorized_input
                    
        

In [44]:
def find_most_similar_tweets(vectorized_input, vectorized_tweets,topn):
    vec_tweets=np.vstack(vectorized_tweets.Vector.apply(lambda x: x.tolist()))
    cos=model.cosine_similarities(vectorized_input['Vector'].reshape(-1,), vec_tweets)
    vectorized_tweets.loc[:,'Similarity_score'] = np.round(cos,10)
    decimals = pd.Series([8], index=['Similarity_score'])
    vectorized_tweets.round(decimals)
    vectorized_tweets = vectorized_tweets.sort_values(by='Similarity_score', ascending=False)
    return vectorized_tweets[0:topn]

In [47]:
topic = 'brexit' #it can be a list of topics,  comman means 'or'
time_limit=20
user_input = 'Canadian retaliatory tariffs lifted as U.S. kills steel aluminum penalties'
def process_user_input(user_input, time_limit, topic):
    data_dir = '/Users/shahla/Dropbox/SharpestMinds/stream/data/'
    track_list=[k for k in topic.split(',')]
    file_name = "stream"
    if os.path.exists(data_dir+file_name+'.json'):
        os.remove(data_dir+file_name+'.json')
    #-----------------------------------------------
    # Load credentials from json file
    with open("twitter_credentials.json", "r") as file:  
         credentials = json.load(file)

    # Instantiate an object
    python_tweets = Twython( credentials['CONSUMER_KEY'],  credentials['CONSUMER_SECRET'])
    auth = tweepy.OAuthHandler(credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'])
    auth.set_access_token(credentials['ACCESS_TOKEN'], credentials['ACCESS_SECRET'])
    latest_tweets, no_data = get_latest_tweets(data_dir, auth, time_limit, track_list)
    if no_data:
        return'There is no data with topic: '+topic+' in the last '+ str(time_limit)
    else:
#       downloaded pretrained model
        model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)
        vectorized_tweets = vectorize_latest_tweets(latest_tweets)
        vectorized_user_input = vectorize_user_input(user_input)
#       find the top topn= 10  similar tweets
        recommendations = find_most_similar_tweets(vectorized_user_input, vectorized_tweets,topn=10)
    return recommendations
    

In [ ]:
topic = 'brexit' #it can be a list of topics,  comman means 'or'
time_limit=20
user_input = 'Canadian retaliatory tariffs lifted as U.S. kills steel aluminum penalties'
recommendations = process_user_input(user_input, time_limit, topic)